In [55]:
import dash
from dash import dcc, html, callback_context
from dash.dependencies import Input, Output, State
import plotly.graph_objs as go

from controllers.train_controller import train_model
from controllers.test_controller import test_model
from models.src.db_manager import DBManager

db = DBManager("work.db")

###############################################################################
# Класс-обертка
###############################################################################
class Experiments:
    def __init__(self):
        self.metrics = {'RMSE': [], 'MAE': []}

    def run(self, model_type, warm_start, start, end):
        print(model_type, warm_start, start, end)
        train_model(
            model_type=model_type,
            warm_start=warm_start,
            db_path="work.db",
            start=start,
            end=end
        )
        rmse, mae = test_model("latest", "test.csv")
        self.metrics['RMSE'].append(rmse)
        self.metrics['MAE'].append(mae)
        return self.metrics

exp = Experiments()

###############################################################################
# Dash UI
###############################################################################
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Dash-интерфейс для обучения модели"),

    html.Div([
        html.Label("Выберите тип модели:"),
        dcc.Dropdown(
            id='model-type-dropdown',
            options=[
                {'label': 'Linear', 'value': 'LR'},
                {'label': 'Ridge', 'value': 'Ridge'},
                {'label': 'RandomForest', 'value': 'RF'}
            ],
            value='ridge'
        )
    ], style={'margin': '10px'}),

    html.Div([
        html.Label("Warm start:"),
        dcc.RadioItems(
            id='warm-start-radio',
            options=[
                {'label': 'Да', 'value': True},
                {'label': 'Нет', 'value': False}
            ],
            value=True,
            labelStyle={'display': 'inline-block', 'margin-right': '10px'}
        )
    ], style={'margin': '10px'}),

    html.Div([
        html.Label("Диапазон индексов (start, end):"),
        dcc.RangeSlider(
            id='range-slider',
            min=0,
            max=db.get_length(),
            step=1,
            value=[0, min(100, db.get_length())],
            allowCross=False,
            marks={i: str(i) for i in range(0, db.get_length()+1, max(1, db.get_length()//10))}
        )
    ], style={'margin': '10px'}),

    html.Button("Обучить модель", id='train-button', n_clicks=0),

    dcc.Graph(id='metrics-graph')
])

###############################################################################
# Callbacks
###############################################################################
@app.callback(
    Output('metrics-graph', 'figure'),
    Input('train-button', 'n_clicks'),
    State('model-type-dropdown', 'value'),
    State('warm-start-radio', 'value'),
    State('range-slider', 'value')
)
def train_and_update_graph(n_clicks, model_type, warm_start, range_vals):
    if n_clicks == 0:
        return go.Figure()

    start, end = range_vals
    metrics = exp.run(model_type, warm_start, start, end)

    fig = go.Figure()
    fig.add_trace(go.Scatter(y=metrics['RMSE'], mode='lines+markers', name='RMSE'))
    fig.add_trace(go.Scatter(y=metrics['MAE'], mode='lines+markers', name='MAE'))
    fig.update_layout(title="Метрики на тесте", xaxis_title="Итерация", yaxis_title="Значение")
    return fig

if __name__ == '__main__':
    app.run(debug=True)


<IPython.core.display.Javascript object>

### Пример работы дашборда:
- Выбором модели
- Дообучение
- Выбор среза базы данных, на котором проводим обучение


![](example_of_dash.png)